In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install lime
import sklearn
import sklearn.ensemble
from __future__ import print_function
np.random.seed(1)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import lime
import lime.lime_tabular

np.random.seed(1)

In [ ]:
iris_df = pd.read_csv("iris.csv")
print(iris_df.shape)
iris_df.head(5)

This dataset contains three iris species: setosa, versicolor, and virginica. It also contains the flower's petal length, petal width, sepal length, and sepal width. Our goal is to use logistic regression so that we can predict what species of iris we have given its measurements.

In [ ]:
X = iris_df[["SepalLengthCm",  "SepalWidthCm",  "PetalLengthCm",  "PetalWidthCm"]]
Y = iris_df["Species"]

In [ ]:
import sklearn
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(Y)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

pipeline = Pipeline([('scalar', StandardScaler()),('classifier', LogisticRegression(max_iter=200))])

In [ ]:
pipeline.fit(X_train, Y_train)

NameError: name 'pipeline' is not defined

In [ ]:
Y_pred = pipeline.predict(X_test)
print(X_test)
print("\nY_pred: ", Y_pred)

In [ ]:
import sklearn
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(Y_test, Y_pred)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import sklearn
from sklearn.metrics import classification_report
class_report = classification_report(Y_test, Y_pred)
print("\nClassification Report:\n", class_report)

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Train Random Forest model
rf_model = RandomForestClassifier(n_estimators=500, random_state=73)
rf_model.fit(X_train_scaled, Y_train)

RandomForestClassifier(n_estimators=500, random_state=73)

In [ ]:
Y_pred = rf_model.predict(X_test_scaled)

print("X_test subset contains 30 random flowers and their metrics):")
print(X_test_scaled)

print("\nY_pred is an array of 30 predictions: [0:setosa, 1:versicolor, 2:verginica]")
print(Y_pred)

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(
    X_train_scaled,
    feature_names=X.columns,
    class_names=iris.target_names,
    mode='classification'
)

In [ ]:
instance = X_test_scaled[15]
print(instance)

exp = explainer.explain_instance(instance, rf_model.predict_proba, num_features=4)

# Visualize explanation
exp.as_pyplot_figure()
plt.show()

# Print feature importance
print(exp.as_list())

The visualisation explains why it the chosen instance of flower from the test data is predicted to be from the species "versicolor". Using the input features such as petal width, petal legth, sepal length and sepal width, it has determined the given flower to be of versicolor due to its similarity with other flowers with similar measurements being of the versicolor species. The strongest similarieties were of petal width and petal length, which strongly agree with the "versicolor" species petal width and length measurements, While sepal width does not fully agree with the prediction, it does not have much of an impact compared to the other input features.Sepal width, while having minor impact on the prediction, it still agrees witht he prediction unlike sepal width which does not agree with the prediction of the species as "versicolor"